**Métodos computacionales 2**

**Universidad de los Andes**

In [10]:
import time
import numpy as np
import scipy as sci
import scipy.integrate as integrate

import matplotlib.pyplot as plt
from matplotlib import animation
plt.style.use('dark_background')

In [11]:
from matplotlib import rc
rc("animation", html = "jshtml")

## Atracción gravitacional Newtoniana

$$\vec{F_{21}} =m_1 \frac{d^2\vec{r}_1}{dt^2}= G \frac {m_{1}m_{2}} {r_{21}^3}\vec{r}_{21}$$

Esta ecuación de segundo grado se puede separar en dos ecuaciones de primer grado y se puede generalizar para la atracción entre un par de cuerpos $i$ y $j$

$$m_i \frac{d\vec{v}_i}{dt}= G \frac {m_{i}m_{j}} {r_{ji}^3}\vec{r}_{ji}$$

$$\frac{d\vec{r}_{i}}{dt}=\vec{v}_i$$

Es conveniente definir un par de constantes que permitan realizar todos los calculos respecto a cantidades de referencia como la masa del sol o la velocidad de la tierra en su orbita al rededor del sol. Con lo anterior se evita realizar continuamente calculos con cantidades que se encuentran en ordenes de magnitud muy dispares.

$$ \boxed{\frac{d\vec{v}_i}{dt}= K_1 \frac {m_{j}} {r_{ji}^3}\vec{r}_{ji}}$$

$$\boxed{\frac{d\vec{r}_i}{dt}=K_2\vec{v}_i}$$


La velocidad y posición del centro de masa se puede calcular como

$$\vec{r}_{cm} = \frac{\sum^{n}_{i=1}m_i\vec{r}_i}{\sum^{n}_{i=1}m_i}$$

$$\vec{v}_{cm} = \frac{\sum^{n}_{i=1}m_i\vec{v}_i}{\sum^{n}_{i=1}m_i}$$



## Solución numérica del problema de los 3 cuerpos

### Parámetros y condiciones iniciales

In [12]:
#Constante de gravitación universal
G = 6.67408e-11 # m^3 / (s^2 kg)

#cantidades de referencia
masa_ref = 1.989e+30 # kg
dist_ref = 5.326e+12 # m
velo_ref = 30000 # m/s
time_ref = 400*365*24*3600 # s

#masas
m1 = 1.1 
m2 = 0.907 
m3 = 1.0 

#posiciones iniciales
r1 = np.array([-0.5, 0.0, 0.0])
r2 = np.array([ 0.5, 0.0, 0.0])
r3 = np.array([ 0.0, 1.0, 0.0])

#velocidades iniciales
v1 = np.array([0.2, 0.02, 0.0])
v2 = np.array([-0.01, 0,-0.11])
v3 = np.array([0.0,-0.05 ,0.1])

# num puntos trayectoria
puntos = int(10 * time_ref/(365*24*3600)) 

#parámetros animación
fps = 48 # fotogramas por segundo
tiempo_anim = int(time_ref/(20*365*24*3600)) # duración de la animación en segundos

#constantes
K1 = G*time_ref*masa_ref/(dist_ref**2*velo_ref)
K2 = velo_ref*time_ref/dist_ref


### Ecuaciones diferenciales

In [13]:
def tres_cuerpos(t, w, G, m1, m2, m3):
  r1, r2, r3, v1, v2, v3 = w.reshape(6, 3)
  r12 = np.linalg.norm(r2 - r1)
  r13 = np.linalg.norm(r3 - r1)
  r23 = np.linalg.norm(r3 - r2)
  dv1_dt = K1*m2*(r2 - r1)/r12**3 + K1*m3*(r3 - r1)/r13**3
  dv2_dt = K1*m2*(r2 - r1)/r12**3 + K1*m3*(r3 - r1)/r13**3
  dv3_dt = K1*m2*(r2 - r1)/r12**3 + K1*m3*(r3 - r1)/r13**3
  dr1_dt = K2*v1
  dr2_dt = K2*v2
  dr3_dt = K2*v3
  return derivs

In [14]:
def tres_cuerpos(t, w, G, m1, m2, m3):
  r1, r2, r3, v1, v2, v3 = w.reshape(6,3)
  r12 = np.linalg.norm(r2 - r1)
  r13 = np.linalg.norm(r3 - r1)
  r23 = np.linalg.norm(r3 - r2)
  dv1_dt = K1*m2*(r2 - r1)/r12**3 + K1*m3*(r3 - r1)/r13**3
  dv2_dt = K1*m1*(r1 - r2)/r12**3 + K1*m3*(r3 - r2)/r23**3
  dv3_dt = K1*m1*(r1 - r3)/r13**3 + K1*m2*(r2 - r3)/r23**3
  dr1_dt = K2*v1
  dr2_dt = K2*v2
  dr3_dt = K2*v3
  derivs = np.concatenate([dr1_dt, dr2_dt, dr3_dt, dv1_dt, dv2_dt, dv3_dt])
  return derivs

### Solucionar ecuaciones

In [15]:
# transforma las velocidades (o posiciones) al sistema centro de masa
def transformar_al_cm(arr_m, arr_v):
  v_cm = (arr_m @ arr_v)/np.sum(arr_m)
  return arr_v - np.array([v_cm,]*len(arr_v))

def solucion_ecuaciones(cond_ini, puntos):
  t = np.linspace(0, 1, puntos)
  args = [G, m1, m2, m3]
  sol = integrate.solve_ivp(tres_cuerpos,[0,1],cond_ini,'DOP853',t,args=args)
  s = sol.y
  return np.array([s[:3, :], s[3:6, :], s[6:9, :]]) # [body1, body2, body3]

v1cm, v2cm, v3cm = transformar_al_cm(np.array([m1,m2,m3]),np.array([v1,v2,v3]))
cond_ini = np.array([r1, r2, r3, v1cm, v2cm, v3cm]).flatten()
data = solucion_ecuaciones(cond_ini, puntos)
data.shape

(3, 3, 4000)

### Mostrar solución

In [16]:
def update(num_frames, data, elements, paso):
  j = num_frames * paso
  lines, bodies = elements
  bodies._offsets3d = (data[:, 0, j], data[:, 1, j], data[:, 2, j])
  for i in range(0, 3):
    lines[i].set_data_3d(data[i,:, :j+1])

def ajustar_grafica(data):
  fig = plt.figure(figsize = (16,9), dpi = 75)
  fig.subplots_adjust(left=0, right=1, bottom=0, top=1) #margen
  #ax = fig.gca(projection='3d')
  ax = fig.add_subplot(projection='3d')
  ax.view_init(elev=20, azim=90)    # punto de vista
  ax.set_xlim3d([np.min(data[:,0,:]), np.max(data[:,0,:])])
  ax.set_ylim3d([np.min(data[:,1,:]), np.max(data[:,1,:])])
  ax.set_zlim3d([np.min(data[:,2,:]), np.max(data[:,2,:])])
  ax._axis3don = False    # ocultar ejes
  ax.dist = 7    # controlar la distancia entre los ejes y la camara
  return fig, ax

def inicializar_elementos_graficos(d, ax):
  line1, = ax.plot(d[0,0,:1],d[0,1,:1],d[0,2,:1],color='#ff3030')
  line2, = ax.plot(d[1,0,:1],d[1,1,:1],d[1,2,:1],color='#24ff24')
  line3, = ax.plot(d[2,0,:1],d[2,1,:1],d[2,2,:1],color='#1f75ff')
  bodies = ax.scatter(d[:,0,0],d[:,1,0],d[:,2,0],s=200,c='y',alpha=1)
  return [[line1, line2, line3],bodies]

def crear_animacion(data,time,fps,name):
  N = time*fps
  paso = int(np.floor(data.shape[2] / N))
  fig, ax = ajustar_grafica(data)
  elements = inicializar_elementos_graficos(data, ax)
  ani = animation.FuncAnimation(fig, update, N, fargs=(data,elements,paso))
  # return ani
  ani.save(name,fps=fps)
  plt.close(fig)

start_time = time.time()
crear_animacion(data,tiempo_anim,fps,'3cuerpos.gif')
#print("Tiempo usado en crear la animación (s):",time.time() - start_time)

/var/folders/0k/fbxvx_0d4kb5fjxprl22wdtm0000gn/T/ipykernel_96303/4047781034.py:18: MatplotlibDeprecationWarning: The dist attribute was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax.dist = 7    # controlar la distancia entre los ejes y la camara
MovieWriter ffmpeg unavailable; using Pillow instead.


In [17]:
print("Tiempo usado en crear la animación (s):",time.time() - start_time)

Tiempo usado en crear la animación (s): 20.595019102096558


In [18]:
# Mostrar video
from IPython.display import HTML
from base64 import b64encode

name = '3cuerpos.mp4'
mp4 = open(name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

FileNotFoundError: [Errno 2] No such file or directory: '3cuerpos.mp4'

**Referencias**

Profesores: Diego Alberto Castro, Metodos Computacionales Uniandes